DataJoint U24 - Workflow Volume


### Intro


This notebook will describe the steps to use Element Volume for interacting with BossDB.
Prior to using this notebook, please refer to documentation for
[Element installation instructions](https://datajoint.com/docs/elements/user-guide/) and refer to [BossDB resources](https://www.youtube.com/watch?v=eVNr6Pzxoh8) for information on generating an account and configuring `intern`.

Importantly, you'll need an `intern` config file, which should look like this:

```cfg
    # ~/.intern/intern.cfg
    [Default]
    protocol = https
    host = api.bossdb.io
    token = <YOUR_TOKEN>
```


In [1]:
import datajoint as dj
import os

if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir("..")
dj.conn()

[2023-01-30 15:47:26,090][INFO]: Connecting cbroz@dss-db.datajoint.io:3306
[2023-01-30 15:47:26,388][INFO]: Connected cbroz@dss-db.datajoint.io:3306


DataJoint connection (connected) cbroz@dss-db.datajoint.io:3306

In [2]:
dj.config["custom"]["database.prefix"] = "cbroz_wfboss_"
dj.config["custom"][
    "vol_root_data_dir"
] = "/Users/cb/Documents/data/U24_SampleData/boss/"
from workflow_volume.pipeline import volume, BossDBInterface

volume.Volume.delete_quick()

[2023-01-30 15:47:26,504][WARNING]: lab.Project and related tables will be removed in a future version of Element Lab. Please use the project schema.


`BossDBInterface` works much like `intern.array`, but with additional functionality for managing records in your Element Volume schema. We can optionally link this dataset to a session in our pipeline via a session key.

Note, however, that we'll have to change our notation slightly. Whereas we can directly index into a dataset to get slices, we'll need to either provide slices as a string or a tuple.


### Testing


In [3]:
data = BossDBInterface(
    "bossdb://takemura/takemura13/image", resolution=4, session_key={}
)

/Users/cb/Documents/dev/intern/intern/convenience/array.py:922: ImportWarning: CloudVolume is not installed. Accessing channel using CVDB.
  warnings.warn(


Using `intern` notion, we can look at Z slice 300, from Y voxels 200-500, and X voxels 0 to 700.


In [4]:
data[300, 200:501, 0:701]

array([[139, 152, 129, ..., 160, 125, 154],
       [148, 146, 127, ..., 134, 118, 170],
       [ 75, 173, 145, ..., 107, 115, 111],
       ...,
       [ 98,  99,  99, ..., 119,  93,  87],
       [ 91,  86,  88, ...,  86,  79,  84],
       [118, 130, 102, ...,  96,  88, 111]], dtype=uint8)

The same data can be downloaded and loaded into Element Volume using either of the following commands.

Without a session directory provided via `get_session_directory` in `workflow_volume.paths`, we will infer an output directory based on the BossDB path from `get_vol_root_data_dir`.


In [5]:
# data.download(slice_key=(300,slice(200,501),slice(0,701)))
data.download(slice_key="[300,200:501,0:701]")

Our volume is stored in the `Volume`

In [7]:
volume.Volume()

volume_id shorthand for this volume,"resolution_id Shorthand for convention. For BossDB, integer value.",subject,session_datetime,z_size total number of voxels in z dimension,y_size total number of voxels in y dimension,x_size total number of voxels in x dimension,slicing_dimension perspective of slices,"channel data type or modality (e.g., EM, segmentation, etc.)",url dataset URL
takemura/takemura13,4,None,None,1460,750,750,z,image,https://api.bossdb.io/v1/mgmt/resources/takemura/takemura13/image


With `Slice` corresponding to slices

In [8]:
volume.Volume.Slice()

volume_id shorthand for this volume,"resolution_id Shorthand for convention. For BossDB, integer value.",id Nth voxel in slicing_dimension,zoom_id Shorthand for zoom convention,file_path filepath relative to root data directory
takemura/takemura13,4,300,X0-701_Y200-501,takemura/takemura13/image/Res4_ZoomX0-701_Y200-501_Z300.png


Each BossDB resolution will have a unique entry in the `Resolution` table

In [9]:
volume.Resolution()

"resolution_id Shorthand for convention. For BossDB, integer value.","voxel_unit e.g., nanometers",voxel_z_size size of one z dimension voxel in voxel_units,voxel_y_size size of one y dimension voxel in voxel_units,voxel_x_size size of one x dimension voxel in voxel_units
4,nanometers,70.4,70.4,704.0


And, the `Zoom` table retain information about the X/Y windows we use.

In [10]:
volume.Zoom()

zoom_id Shorthand for zoom convention,first_start Starting voxel in first dimension (X if taking Z slices),first_end Ending voxel plus 1 in first dimension,second_start Starting voxel in second dimension (Y if taking Z slices),second_end Ending voxel plus 1 in second dimension
Full Image,0,None,0,None
X0-701_Y200-501,0,701,200,501


Changing any of these pieces of information would download different data.

In [12]:
data.download(slice_key=(slice(300, 311), slice(100, 401), slice(100, 401)))